## Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as k
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from keras.models import model_from_json

## Globals

In [ ]:
#Use GPU
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

CLASS_LABELS = ['non-violent','violent']
LABELS_COUNT = len(CLASS_LABELS)

DATASET_PATH = 'Dataset/violent_keypoints_dataset.csv'
CONF_DATASET_PATH = 'Dataset/violent_keypoints_conf_dataset.csv'

MEAN_CONF_THRESHOLD = 0.3

TRAIN_SPLIT = 0.8

EPOCHS = 50
BATCH_SIZE = 32

## Import Dataset

In [ ]:
#Import Dataset
data = pd.read_csv(DATASET_PATH).values

data[data == "non-violent"] = 0
data[data == "violent"] = 1

data = data.astype('float32')
data_no_conf = data

print(data.shape)
print("-----DATA-----")
print("Non-Violent: " + str(np.count_nonzero(data[:,0] == 0)))
print("Violent: " + str(np.count_nonzero(data[:,0] == 1)))

## Import Confidence Scores

In [ ]:
#Import Confidence
conf_data = pd.read_csv(CONF_DATASET_PATH).values

conf_data = conf_data[:, 1:]

conf_data = conf_data.astype('float32')

print(conf_data)
print(conf_data.shape)

## Remove Data w/ Low Confidence

In [ ]:
high_conf_data = []

count = 0

#Mean Confidence
for x in range(len(conf_data)):
    count += np.mean(conf_data[x])
    if np.mean(conf_data[x]) >= MEAN_CONF_THRESHOLD:
        high_conf_data.append(data[x])

avg_mean = count / len(conf_data)
print("AVG MEAN: " + str(avg_mean))

data = np.array(high_conf_data)


print(data)
print(data.shape)

## Split Dataset

In [ ]:
TRAIN_AMOUNT = int(data.shape[0] * TRAIN_SPLIT)
RANDOM_SEED = 89

#Shuffle Data
for x in range(100):
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(data)

#Split Data into Train and Test
train = data[0:TRAIN_AMOUNT]
test = data[TRAIN_AMOUNT:]

#Extract Data and Reshape to (3,38)
x_train = np.reshape(train[:, 1:],(train.shape[0],19,3,2)).swapaxes(2,3).swapaxes(1,3)
x_train = np.reshape(x_train,(train.shape[0],3,38), order="F")
x_test = np.reshape(test[:, 1:],(test.shape[0],19,3,2)).swapaxes(2,3).swapaxes(1,3)
x_test = np.reshape(x_test,(test.shape[0],3,38), order="F")

#Extract Labels
y_train = train[:,0]
y_test = test[:,0]

print("-----TRAIN-----")
print("Non-Violent: " + str(np.count_nonzero(y_train == 0)))
print("Violent: " + str(np.count_nonzero(y_train == 1)))

print("-----TEST-----")
print("Non-Violent: " + str(np.count_nonzero(y_test == 0)))
print("Violent: " + str(np.count_nonzero(y_test == 1)))

print("-----DATA-----")
print(data)

## Build CNN Model

In [ ]:
my_model = Sequential()

my_model.add(Conv1D(64, 3, padding="same", activation="relu", input_shape=(3,38)))
my_model.add(Conv1D(64, 3, padding="same", activation="relu"))

my_model.add(Dropout(0.1))

my_model.add(LSTM(38, return_sequences=True))

my_model.add(MaxPooling1D(3, padding='same'))

my_model.add(Flatten())

my_model.add(Dropout(0.1))

my_model.add(Dense(128, activation="relu"))

my_model.add(Dropout(0.5))

my_model.add(Dense(1, activation="sigmoid"))

## Train Model

In [ ]:
# Compile Model
my_model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])

#Patience Callback
patienceCallback = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)]

#Train Model
history = my_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data = (x_test, y_test), callbacks=patienceCallback)

## Model Evaluation

In [ ]:
#Evaluate Model
score = my_model.evaluate(x_test, y_test, batch_size=EPOCHS)

my_model.summary()

## Plot Graphs

In [ ]:
plt.style.use('fivethirtyeight')

xepochs = [i+1 for i in range(0, len(history.history['loss']))]
plt.figure(figsize=(5,3))

# Loss
plt.plot(xepochs, history.history['loss'])
plt.plot(xepochs, history.history['val_loss'])
plt.xticks(xepochs)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

# Accuracy
plt.figure(figsize=(5,3))
plt.plot(xepochs, history.history['accuracy'])
plt.plot(xepochs, history.history['val_accuracy'])
plt.xticks(xepochs)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

## Classification Report

In [ ]:
#Shuffle Data
for x in range(100):
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(data_no_conf)

test = data_no_conf[TRAIN_AMOUNT:]

x_test = np.reshape(test[:, 1:],(test.shape[0],19,3,2)).swapaxes(2,3).swapaxes(1,3)
x_test = np.reshape(x_test,(test.shape[0],3,38), order="F")

y_test = test[:,0]

predictions = my_model.predict(x_test)

pred_class = (predictions > 0.5).astype(int)

print(classification_report(y_test, pred_class, target_names=['non-violent', 'violent']))

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, pred_class)
print(cm)

## Export Model and Weights

In [ ]:
# serialize to JSON
json_file = my_model.to_json()
with open("ViolentModel/CNN/violent_model_json", "w") as file:
   file.write(json_file)

# serialize weights to HDF5
my_model.save_weights("ViolentModel/CNN/violent_model_weights.h5")